<a href="https://colab.research.google.com/github/natdebandi/hate_speech_ar/blob/main/prueba_modelos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Prueba de modelos

In [1]:
! pip install -U accelerate
! pip install -U transformers

In [2]:
!pip install datasets seaborn

In [3]:
import torch
torch.cuda.is_available()

False

In [4]:
from transformers import pipeline

In [5]:
examples = ["hay que matarlos a todos", "Vamos a la plaza a marchar","Los chinos son todos sucios","Culpa de los bolitas"]

### Modelo 1 BETO binario

Parametros:

output_dir=repo_name,
eval_strategy ="steps",
adam_beta1=0.9,
adam_beta2=0.98,
eval_steps=10000,
per_device_train_batch_size=16,
per_device_eval_batch_size=16,
weight_decay=0.01,
push_to_hub=True,

In [22]:
specific_model = pipeline(model="natdebandi/beto_hate_speech_ar_bin")
specific_model(examples)

config.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/439M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.24k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/248k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/735k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

[{'label': 'LABEL_1', 'score': 0.9779065847396851},
 {'label': 'LABEL_0', 'score': 0.9912819862365723},
 {'label': 'LABEL_1', 'score': 0.9802582263946533},
 {'label': 'LABEL_1', 'score': 0.9858723282814026}]

### Modelo 2 BETO multietiqueta




In [20]:
from transformers import AutoTokenizer,AutoModelForSequenceClassification

checkpoint = "natdebandi/beto_hate_speech_ar_multiclasif"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

model = AutoModelForSequenceClassification.from_pretrained(checkpoint)

tokens = tokenizer(examples, padding=True, truncation=True, return_tensors="pt")
outputs = model(**tokens)

outputs

SequenceClassifierOutput(loss=None, logits=tensor([[ 4.4980,  4.8614, -4.7389, -5.0946, -0.4573, -3.1337, -3.9628, -5.4577,
         -5.1387,  1.0960],
        [-5.5471, -7.7568, -7.6834, -7.4151, -7.5128, -7.0103, -6.8649, -8.3965,
         -8.0500, -6.7079],
        [ 4.7861, -5.1375, -4.4678, -4.6567,  3.7283, -4.8593, -5.0861, -4.8315,
         -0.1001, -5.1888],
        [ 3.9923, -5.9076, -4.9183, -5.6641,  3.8996, -5.3985, -5.1954, -5.8583,
         -5.4512, -5.4603]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [ ]:
# Process the results as needed, for example:
for result in results:

  predicted_labels = [id2label[idx] for idx, score in enumerate(result) if score > 0.5]  # Adjust threshold as needed
  print(predicted_labels)

In [21]:
logits = outputs.logits
logits.shape

torch.Size([4, 10])

In [22]:
import numpy as np

# apply sigmoid + threshold
sigmoid = torch.nn.Sigmoid()
probs = sigmoid(logits.squeeze().cpu())
predictions = np.zeros(probs.shape)
predictions[np.where(probs >= 0.5)] = 1


In [23]:
model.config.id2label

{0: 'HATEFUL',
 1: 'CALLS',
 2: 'WOMEN',
 3: 'LGBTI',
 4: 'RACISM',
 5: 'CLASS',
 6: 'POLITICS',
 7: 'DISABLED',
 8: 'APPEARANCE',
 9: 'CRIMINAL'}

In [24]:
predictions

array([[1., 1., 0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 1., 0., 0., 0., 0., 0.]])

In [26]:
# prompt: apply model.config.id2label to the predictions to get the labes for all the values

predicted_labels = []
for prediction in predictions:
  example_labels = [model.config.id2label[idx] for idx, value in enumerate(prediction) if value == 1]
  predicted_labels.append(example_labels)

predicted_labels


[['HATEFUL', 'CALLS', 'CRIMINAL'],
 [],
 ['HATEFUL', 'RACISM'],
 ['HATEFUL', 'RACISM']]